In [3]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import * 

from numpy import *

ERROR:root:The 'ast_node_interactivity' trait of a ZMQInteractiveShell instance must be any of ['all', 'last', 'last_expr', 'none'], but a value of 'last_expr_or_assign' <type 'str'> was specified.


SyntaxError: invalid syntax (modsim.py, line 1107)

# Binary System of Sirius

---
## Question:
### How will the orbit be like if Sirius B experiences a sudden accleration?
---

We pick a binary star system that we are most familiar with - our neighbor Sirius A and Sirius B. Most stars have a companion, the gravitational force, serving as the centripital force, keeps them moving in orbit around the center of mass. When a star accelarates, the centripetal force it needs to keep in orbit would increase, so the gravitational force is not adequate to keep the star in its original orbit, so the star would actually leave its companion. However, if we assume there is friction or radiation -- something that wastes the kineetic energy, the velocity would go down and the star would be dragged back to its companion. In this model, we would simulate the orbit when Sirius B tries to escape and get pulled back.

In [4]:
# Units 
s = UNITS.second
N = UNITS.newton
kg = UNITS.kilogram
m = UNITS.meter


NameError: name 'UNITS' is not defined

In [ ]:
help.UNITS